In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [6]:
#### csv 整理
def dataClean(city):
    # 取得所有 CSV 檔案的路徑
    csv_files = glob.glob(f'agodaHotelComments_{city}/*.csv')

    ### 修改 csv (只保留需要的欄位)
    # 建立一個包含所有想要保留的列名的列表
    columns_to_keep = ["所有客房均提供", "整體狀況及整潔度", "評論分數", "總評", "評論者", 
                       "評論者國籍", "入住人數", "房型", "住宿細節", "評論標題", 
                       "評論副標題", "評論內容", "評論日期", "地址", "所有房型", 
                       "附近景點", "飯店整體分數", "飯店整體評價", "已驗證評論數", 
                       '可使用語言', '友善設施', '網路服務', '休閒娛樂設施', '衛生安全', 
                       '服務與便利設施', '接待設施', '設施與設備', '位置', '服務', 'CP值', 
                       '餐飲服務', '交通服務/設施', '兒童專屬', '客房舒適度', '安全與衛生', 
                       '衛浴', '戶外設施', '標準設施', '廚房', '娛樂', '舒適設備', '衛浴設備', 
                       '格局與擺設', '衣物與洗衣設備', '安全特色', '運動器材租用', '親子', 
                       'Restaurant', '陸上運動']


    for file in csv_files:
        # 讀取每個 CSV 檔案
        data = pd.read_csv(file, header=0)

        # 刪除不在列名列表中的所有欄位，然後將修改過的資料存回原本的 CSV 檔案
        columns_to_drop = [col for col in data.columns if col not in columns_to_keep]
        data.drop(columns_to_drop, axis=1, inplace=True)

        # 新增飯店名稱欄位
        filename = os.path.splitext(os.path.basename(file))[0]  # 獲取檔名（不含路徑和副檔名）
        data['飯店名稱'] = filename

        # 新增縣市欄位
        data['縣市'] = city

        # 儲存回原本的 csv
        data.to_csv(file, index=False, encoding='utf-8-sig')

#### 合併 csv(縣市)
def merge(city):
    # 取得所有 CSV 檔案的路徑
    csv_files = glob.glob(f'agodaHotelComments_{city}/*.csv')

    df = pd.DataFrame()

    for file in csv_files:
        data = pd.read_csv(file, header=0, low_memory=False)
        df = pd.concat([df, data], axis=0,ignore_index=True)

    df.to_csv(f'agodaHotelComments_{city}.csv', index=False, encoding='utf-8-sig')
    
#### 合併全部資料
def totalmerge():
    # 取得所有 CSV 檔案的路徑
    csv_files = glob.glob(f'agodaHotelComments_*.csv')
    
    df = pd.DataFrame()

    for file in csv_files:
        data = pd.read_csv(file, header=0, low_memory=False)
        df = pd.concat([df, data], axis=0,ignore_index=True)

    df.to_csv('agodaHotelComments_all.csv', index=False, encoding='utf-8-sig')

In [3]:
if __name__ == '__main__':
    
    citys = ['台中市','台北市','台東縣','台南市','宜蘭','花蓮縣','金門縣','南投','屏東縣','苗栗','桃園市','馬祖','高雄市','雲林','新竹','嘉義縣','彰化','綠島','澎湖縣','墾丁','基隆市']
    for city in citys:
        dataClean(city)
        merge(city)

In [10]:
if __name__ == '__main__':
    totalmerge()

In [3]:
df = pd.read_csv('agodaHotelComments_all.csv', header=0)
unique_values = df['飯店名稱'].unique()
print(len(unique_values))

C:\Users\student\AppData\Local\Temp\ipykernel_8760\561990364.py:1: DtypeWarning: Columns (34,36,37,39,40,41,42,43,44,45,46,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('agodaHotelComments_all.csv', header=0)


6408


In [ ]:
# 把文字左右兩側的空白去除
df['評論內容'] = df['評論內容'].apply(lambda x: x.strip() if pd.notnull(x) else x)
df['評論副標題'] = df['評論副標題'].apply(lambda x: x.strip() if pd.notnull(x) else x)
df['評論標題'] = df['評論標題'].apply(lambda x: x.strip() if pd.notnull(x) else x)

# 新增一欄綜合評論
df['綜合評論'] = pd.Series()

# 把 NA 值填空字串，才能將 df['綜合標題'] 的內容併到綜合評論
df['綜合評論'].fillna('', inplace=True)

# 計算評論標題
df_comments_title_count = df['評論標題'].value_counts();df_comments_title_count

# 計算評論副標題
df_comments_subTitle_count = df['評論副標題'].value_counts();df_comments_subTitle_count

# 因為有遊客會把評論寫在評論標題裡面，所以要把這些留言回填到綜合評論
filter1 = df['評論標題'].map(df['評論標題'].value_counts()) < 14000
df.loc[filter1, '綜合評論'] += df.loc[filter1, '評論標題'] + '。'

# 因為有遊客會把評論寫在評論副標題裡面，所以要把這些留言回填到綜合評論
filter2 = df['評論副標題'].map(df['評論副標題'].value_counts()) < 50
df.loc[filter2, '綜合評論'] += df.loc[filter2, '評論副標題'] + '。'

# 將評論內容併到綜合評論裡
filter3= df['評論內容'].notnull()
df.loc[filter3,'綜合評論'] += df.loc[filter3,'評論內容']

# 將綜合評論空字串變 NAN
df['綜合評論'].replace('', np.nan, inplace=True);df

df[df['綜合評論'].isna()]

In [8]:
# # 計算 '評論內容' 欄位中 NaN 的數量
# num_nan = df['評論內容'].isna().sum()
# print(f"Number of NaN values: {num_nan}")

# # 計算 '評論內容' 欄位中空字符串的數量
# num_empty_strings = (df['評論內容'] == '').sum()
# print(f"Number of empty strings: {num_empty_strings}")

# 刪除 '綜合評論' 為 NaN 的資料
df.dropna(subset=['綜合評論'], inplace=True)

df[df['綜合評論'].isna()]

,評論分數,總評,評論者,評論者國籍,入住人數,房型,住宿細節,評論標題,評論副標題,評論內容,...,安全特色,安全與衛生,格局與擺設,衣物與洗衣設備,親子,標準設施,廚房,衛浴,陸上運動,綜合評論


In [9]:
df.to_csv(f'agodaHotelComments_all_clean.csv', index=False, encoding='utf-8-sig')

In [ ]:
df = pd.read_csv('agodaHotelComments_all_clean.csv', header=0, low_memory=False);df

### 清理模型csv不會用到的欄位

In [ ]:
import pandas as pd

df = pd.read_csv('./model_agoda/agoda_roberta_hotel_info_綜合評論.csv',encoding='utf-8-sig')

In [5]:
df.columns

Index(['整體評分', '總評', '評論者', '評論者國籍', '入住人數', '房型', '住宿細節', '評論標題', '評論副標題',
       '評論內容', '評論日期', '地址', '所有房型', '附近景點', '飯店整體分數', '飯店整體評價', '已驗證評論數',
       '可使用語言', '網路服務', '休閒娛樂設施', '衛生安全', '餐飲服務', '服務與便利設施', '兒童專屬', '接待設施',
       '交通服務/設施', '所有客房均提供', '整體狀況及整潔度', '設施與設備', '位置', '服務', 'CP值', '飯店名稱',
       '運動器材租用', '客房舒適度', 'Restaurant', '戶外設施', '友善設施', '娛樂', '舒適設備', '衛浴設備',
       '安全特色', '安全與衛生', '格局與擺設', '衣物與洗衣設備', '親子', '標準設施', '廚房', '衛浴', '陸上運動',
       '綜合評論', '縣市', '鄉鎮'],
      dtype='object')

In [ ]:
df = df[['評論者國籍', '地址', '附近景點', '飯店整體分數','整體狀況及整潔度', '設施與設備', '位置', '服務', 'CP值', '飯店名稱',
         '綜合評論', '縣市', '鄉鎮']];df

In [15]:
df.to_csv('./model_agoda/agoda_roberta_hotel_info_綜合評論.csv', index=False, encoding='utf-8-sig')